In [15]:
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import numpy as np

from numpy import genfromtxt
from sklearn.datasets import load_boston
from tensorflow.python.saved_model.builder import SavedModelBuilder
from tensorflow.python.saved_model.signature_def_utils import build_signature_def
from tensorflow.python.saved_model.signature_constants import REGRESS_METHOD_NAME
from tensorflow.python.saved_model.tag_constants import TRAINING, SERVING
from tensorflow.python.saved_model.utils import build_tensor_info

In [19]:
learning_rate = 0.01
cost_history = np.empty(shape=[1],dtype=float)

X = tf.placeholder(tf.float64,[None,n_dim], name="x")
X_test = tf.placeholder(tf.float64,[None,n_dim], name="x_test")
Y = tf.placeholder(tf.float64,[None,1], name="y")
W = tf.Variable(tf.ones([n_dim,1],dtype=tf.float64), name="w")

init = tf.variables_initializer(tf.global_variables(), name="init")

In [20]:
y_ = tf.matmul(X, W, name="y_hat")
y_preds = tf.matmul(X_test, W, name="y_preds")
cost = tf.reduce_mean(tf.square(y_ - Y))
training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost, name="train")

http://aqibsaeed.github.io/2016-07-07-TensorflowLR/

In [28]:
directory = 'boston_regression'
builder = SavedModelBuilder(directory)

with tf.Session(graph=tf.get_default_graph()) as sess:
    sess.run(init)

    signature_inputs = {
        "x": build_tensor_info(X),
        "x_test": build_tensor_info(X_test),
        "y": build_tensor_info(Y)
    }
    signature_outputs = {
        "out": build_tensor_info(y_preds)
    }
    signature_def = build_signature_def(
        signature_inputs, signature_outputs,
        REGRESS_METHOD_NAME)
    builder.add_meta_graph_and_variables(
        sess, [TRAINING, SERVING],
        signature_def_map={
            REGRESS_METHOD_NAME: signature_def
        },
        assets_collection=tf.get_collection(tf.GraphKeys.ASSET_FILEPATHS))
    builder.save(as_text=False)

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: boston_regression/saved_model.pb
